# YOLOv8 Training – Pothole Detection (SAM3 → YOLO)

Notebook ini digunakan untuk **mendokumentasikan** dan **menjalankan** training YOLOv8 untuk deteksi pothole.

**Catatan penting:**
- Hasil training (weights `best.pt/last.pt`, log, dan artifacts) akan tersimpan otomatis di folder `runs/detect/...`.
- Notebook ini hanya menjalankan perintah dan menyimpan output log di cell.

## Prasyarat
- Sudah berada di environment yang benar (mis. conda env `sam3`)
- Dataset YOLO sudah siap, contoh: `pothole_yolo_5fps/` (berisi `images/` dan `labels/`)
- File `data.yaml` sudah ada, contoh: `pothole_yolo_5fps/data.yaml`


## 1) Cek Environment
Cell ini memastikan CUDA & GPU terdeteksi.

In [ ]:
import torch
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
    print('Torch CUDA version:', torch.version.cuda)
print('Torch version:', torch.__version__)


## 2) Cek Ultralytics (YOLO)
Menampilkan versi YOLO CLI.

In [ ]:
!yolo version


## 3) Ringkasan Dataset (5 FPS)
Menghitung jumlah file train/val (images & labels) dan jumlah label non-empty (positives).

In [ ]:
import os
from pathlib import Path

DATASET_DIR = Path('pothole_yolo_5fps')  # ubah jika dataset kamu berbeda

train_imgs = DATASET_DIR/'images'/'train'
val_imgs   = DATASET_DIR/'images'/'val'
train_lbls = DATASET_DIR/'labels'/'train'
val_lbls   = DATASET_DIR/'labels'/'val'

def count_files(p: Path):
    return len([x for x in p.iterdir() if x.is_file()]) if p.exists() else 0

def count_non_empty_labels(p: Path):
    if not p.exists():
        return 0
    n = 0
    for f in p.iterdir():
        if f.is_file() and f.suffix == '.txt' and f.stat().st_size > 0:
            n += 1
    return n

print('=== Dataset Summary ===')
print('Train images :', count_files(train_imgs))
print('Train labels :', count_files(train_lbls))
print('Train positives (non-empty labels):', count_non_empty_labels(train_lbls))
print('Val images   :', count_files(val_imgs))
print('Val labels   :', count_files(val_lbls))
print('Val positives (non-empty labels):', count_non_empty_labels(val_lbls))


## 4) Training YOLOv8
Gunakan perintah yang sama seperti di terminal.

### Opsi A – Training dari awal
Cocok untuk baseline.

**Catatan:**
- Jika OOM, turunkan `batch` (mis. 16)
- Untuk pothole kecil, bisa coba `imgsz=960` (lebih berat)


In [ ]:
# Ubah data=... sesuai dataset kamu
!yolo detect train \
  data=pothole_yolo_5fps/data.yaml \
  model=yolov8n.pt \
  imgsz=640 \
  epochs=120 \
  batch=32 \
  device=0


### Opsi B – Lanjut training (resume)
Kalau kamu mau melanjutkan dari training sebelumnya, pakai `resume=True` dan `model=.../last.pt`.


In [ ]:
# Contoh resume (comment dulu jika tidak dipakai)
# !yolo detect train \
#   resume=True \
#   model=runs/detect/train3/weights/last.pt \
#   data=pothole_yolo_5fps/data.yaml \
#   imgsz=640 \
#   epochs=180 \
#   batch=32 \
#   device=0


## 5) Lokasi Output Training
Secara default, Ultralytics menyimpan hasil di:

- `runs/detect/train*/weights/best.pt`
- `runs/detect/train*/weights/last.pt`

Untuk melihat folder train terbaru:

In [ ]:
!ls -lah runs/detect | tail -n 50


## 6) Inference (Gambar/Video)
### A) Inference Video
Atur `conf` dan `iou` sesuai kebutuhan (mis. `conf=0.10`, `iou=0.4`).


In [ ]:
# Ganti path model & source sesuai kebutuhan
# !yolo detect predict \
#   model=runs/detect/train4/weights/best.pt \
#   source=VIDEO_BARU.mp4 \
#   conf=0.10 \
#   iou=0.4 \
#   save=True \
#   save_txt=True


### B) Konversi AVI → MP4 (opsional)
Jika hasil predict berupa `.avi`, kamu bisa konversi ke `.mp4` menggunakan `ffmpeg`.


In [ ]:
# Contoh (ubah input/output sesuai nama file)
# !ffmpeg -i input.avi -c:v libx264 -crf 23 output.mp4
